# Fall Detection Classifier Training

#### Author: Ben Posnick

### Create feature vectors for training and testing data

See `preprocessing.py`.

In [1]:
import numpy as np
import pandas as pd
from preprocessing import *

N_CLASSES = 8
N_TEST_SUBJECTS = 4
X_train, y_train = create_feature_vectors("./data/sensor_data_training.csv", False)
X_test4, y_test4 = [], []
for i in range(N_TEST_SUBJECTS):
    file_name = "./data/sensor_data_subject_A0" + str(i+1) + ".csv"
    X, y = create_feature_vectors(file_name, True)
    X_test4.append(X)
    y_test4.append(y)
X_test = np.concatenate(X_test4, axis=0)
y_test = np.concatenate(y_test4, axis=0)

print("\nDistribution of training labels:")
print("Total examples:", y_train.shape[0])
for i in range(N_CLASSES):
    print("Class " + str(i) + ":", sum(y_train == i))
    
print("\nDistribution of testing labels:")
print("Total examples:", y_test.shape[0])
for i in range(N_CLASSES):
    print("Class " + str(i) + ":", sum(y_test == i))
    


Distribution of training labels:
Total examples: 9624
Class 0: 1250
Class 1: 874
Class 2: 1250
Class 3: 1250
Class 4: 1250
Class 5: 1250
Class 6: 1250
Class 7: 1250

Distribution of testing labels:
Total examples: 6745
Class 0: 1001
Class 1: 122
Class 2: 207
Class 3: 2869
Class 4: 1350
Class 5: 721
Class 6: 320
Class 7: 155


### Train and evaluate random forest classifier

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


dt = DecisionTreeClassifier(criterion="entropy", splitter="random", max_depth=32,
                            class_weight="balanced", random_state=0)
rf = BaggingClassifier(base_estimator=dt, n_estimators=256, random_state=0).fit(X_train, y_train)

train_acc = rf.score(X_train, y_train)
test_acc = rf.score(X_test, y_test)
print("Training accuracy:", train_acc)
print("Testing accuracy:", test_acc)
for i in range(N_CLASSES):
    idxs = y_test==i
    label_acc = rf.score(X_test[idxs], y_test[idxs])
    print("Class " + str(i) + " accuracy:", label_acc)


Training accuracy: 1.0
Testing accuracy: 0.6800593031875464
Class 0 accuracy: 0.6643356643356644
Class 1 accuracy: 0.6311475409836066
Class 2 accuracy: 0.6231884057971014
Class 3 accuracy: 0.6556291390728477
Class 4 accuracy: 0.7133333333333334
Class 5 accuracy: 0.6893203883495146
Class 6 accuracy: 0.921875
Class 7 accuracy: 0.5161290322580645


### Save model 

In [3]:
from joblib import dump

model_name = "fall-detector-random-forest"
dump(rf, "./models/" + model_name + ".joblib") 

['./models/fall-detector-random-forest.joblib']